# Redraw of the sensitivity analysis graph

The input data is from `sensitivity_summary.csv`. This file was manually created from the `sensitivity_analysis.csv` file -- that's not ideal. I should be less lazy and build `sensitivity_summary.csv` programmatically from `sensitivity_analysis.csv`...

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import plotly.graph_objects as go

In [ ]:

# Select the different output format settings

PaperPlot = True
# PaperPlot = False
if PaperPlot:
    # output_format = 'IMC'
    output_format = 'IEEE'
else:
    output_format = 'online'

if output_format == 'online':
    font_size_px = 14
    linewidth_px = 512
    landscapewidth_px = 654
    plot_path = None
    
    plot_path = Path('plots')

if output_format == 'IEEE':

    font_size_pt = 7
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 252 - offset  
    landscapewidth_pt = 516 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    out_path = Path('.')

if output_format == 'IMC':
    font_size_pt = 7
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 241 - offset  
    landscapewidth_pt = 506 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    out_path = Path('output/2025_IMC/figures')

# Create the output directory if don't exist
Path(out_path).mkdir(parents=True, exist_ok=True)

# Input data
input_path = Path('.')

# Default plot layout
default_layout = {
    "title":None,
    "width":linewidth_px,
    "height":200,
    "font":{"size":font_size_px},
    "yaxis":{'title':{'font':{'size':font_size_px}}},
    "xaxis":{'title':{'font':{'size':font_size_px}}}
}


In [ ]:
file_name =  'sensitivity_summary.csv'
data = Path(input_path,file_name)
df = pd.read_csv(data)

df['delta_total'] = abs(df['delta_min'] - df['delta_max'])
df.sort_values(by='delta_total', inplace=True,ascending=True)
display(df)


In [ ]:


fig = go.Figure()

for col in ["delta_min","delta_max"]:
    fig.add_trace(go.Bar(
        y=df["parameter"], 
        x=df[col], 
        name=col, 
        orientation='h',
        marker=dict(color='#636EFA'),
        showlegend=False,
        text=np.round(df[col], decimals=1),
        textposition='outside',
    ))

annotation = go.layout.Annotation(
        x=0,
        y=1.1,
        xref="x",
        yref="paper",
        text="Baseline",
        showarrow=False,
        xanchor='center'
    )

# ticktext = [0, 2.6, 4, 8]
ticktext = [0, 2.6, 8]
baseline = 2.6
tickvals = np.subtract(ticktext, baseline)

fig.update_xaxes(
        title="Net impact [gCO2e/task]",
        zeroline=True,  # Ensure there's a zero line for divergence
        zerolinecolor="black",
        # use array tick mode to show that the counts to the left of zero are still positive.
        # this is hard coded; generalize this if you plan to create a function that takes unknown or widely varying data
        tickmode = 'array',     
        tickvals = tickvals,
        ticktext = ticktext,
        range=[-4,9],
)


# Define the custom layout options
custom_layout = dict(
    barmode='relative',  
    width=linewidth_px,
    height=250,
    annotations=[annotation],
    margin=dict(l=0, r=0, t=25, b=40),
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'sensitivity.pdf')